In [1]:
import cv2
from ultralytics import YOLO

# Load models
ppe_model = YOLO("E:/Ziad/University/GP/Vision/best (2).pt")  


In [2]:
# Define required PPE
required_ppe = ["helmet", "vest"]

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Webcam error.")
    exit()

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    ppe_results = ppe_model.predict(source=frame, save=False, verbose=False)
    ppe_result = ppe_results[0]

    # Check if 'person' is detected
    person_detected = any(
        ppe_model.names[int(cls)] == "Person" for cls in ppe_result.boxes.cls
    )

    # Step 2: If a person is detected, run PPE detection
    if person_detected:
        detected_ppe = [
            ppe_model.names[int(cls_id.item() if hasattr(cls_id, "item") else cls_id)].lower()
            for cls_id in ppe_result.boxes.cls
        ]

        # Check for missing PPE
        missing = [item for item in required_ppe if item not in detected_ppe]

        # Draw PPE alert
        status = "All PPE present" if not missing else f"Missing: {', '.join(missing)}"
        color = (0, 255, 0) if not missing else (0, 0, 255)
        cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        # Overlay PPE detections (boxes)
        frame = ppe_result.plot()

    else:
        cv2.putText(frame, "No person detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the result
    cv2.imshow("PPE Monitor", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.
